<a href="https://colab.research.google.com/github/coleterrell97/portfolio/blob/master/test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing and Cleaning Data
The first step in the process is to import the data such that it can be manipulated with Python. This is done using Pandas.

From here, the next objective is to clean/simplify the data as much as possible. This will help limit inaccuracies in the model's predictions.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import math

from sklearn.model_selection import KFold
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense

combined_data = pd.read_csv("./drive/MyDrive/test_task/test_task.csv")

# Data Cleaning Process
The data cleaning process for this data set will include multiple steps. Each of these steps will be described in detail (as well as the rationale behind the step) prior to the execution of the code for that step.

## Step 1: Removing the 'Total Players' Column
This feature is superfluous, as it can be calculated easily from the Team 1 and Team 2 features. Removing this column simply makes the data easier to digest.

In [ ]:
combined_data.drop(columns=["Total Players"], inplace=True)

## Step 2: Converting the Number of Goalkeepers Column to a Binary Feature
There are no instances where one team was allowed a keeper and the other team was not. Teams were either both allowed keepers or neither were. In this case, the data can be simplified by converting all of the 2's in this column to 1's, making it a proper binary feature signifying if keepers were allowed or not.

In [ ]:
combined_data["Goalkeeper"].replace(2,1, inplace=True)

## Step 3: Remove "Not Specified" Column
It is unclear what this column's purpose is. The vast majority of the values present in this column are 0, so deleting this feature entirely should have little impact on the model's behavior.

In [ ]:
combined_data.drop(columns=["Not specified"], inplace=True)

## Step 4: Convert Active and Passive Recovery Features to Binary Features
The minute values stored in these columns can easily be back-calculated by looking at the total number of rest minutes. The omitted feature value in this case would be that the type of recovery is unspecified (when both passive and active features are 0).

In [ ]:
for index, row in combined_data.iterrows():
    if row["P.R."] > 0:
      combined_data.at[index, "P.R."] = 1
    if row["A.R."] > 0:
      combined_data.at[index, "A.R."] = 1

## Step 5: Add Aspect Ratio Column
Based on the information presented in the provided Master's thesis, it seems that the aspect ratio of the pitch is a valuable piece of information that was not directly included in the data set. This is an easy calculation, and it will allow for the removal of superfluous features of length, width, and area of the pitch.

In [ ]:
aspect_ratio = combined_data["Pitch Length"]/combined_data["Pitch Width"]
combined_data.insert(9, "Aspect Ratio", aspect_ratio)

## Step 6: Removing Last Unneccessary Columns
I decided to remove the columns for pitch width, length, and area, because they can all be calculated in reverse using the remaining features.

In [ ]:
combined_data.drop(columns=["Pitch Length", "Pitch Width", "Pitch Area"], inplace=True)

## Step 7: Removing Columns with no Output Feature Values
These columns are of little use to training or testing the model, and filling in these missing values would be almost entirely guesswork.

In [ ]:
for index, row in combined_data.iterrows():
  if row["%HRmax"] == 0 and row["%HRres"] == 0 and row["Blood lactate"] == 0 and row["RPE (CR10)"] == 0:
    combined_data.drop(index, axis = 0, inplace=True)

## Step 8: Data Normalization
To prevent any bias due to feature scaling, I normalized all the data such that all features have a range between 0,1 inclusive.

In [ ]:
combined_data.iloc[:, 0:11]=(combined_data.iloc[:, 0:11]-combined_data.iloc[:, 0:11].min())/(combined_data.iloc[:, 0:11].max()-combined_data.iloc[:, 0:11].min())

# Train vs. Test Sets
The next important step in the process is to divide the combined data set into test and train sets. I will then take roughly 80% of the records for the training set and roughly 20% of the records for the test set. The rows were shuffled prior to input into the model (in Excel) to help eliminate any bias present in their ordering. 

In [ ]:
train_set = combined_data.copy().iloc[:396, :]
test_set = combined_data.copy().iloc[396:, :]

# Filling in Missing Data
The next part of the process is to fill in missing label data. This can be done in multiple ways. The first method I intend to try is to assume a linear relationship between each of the indicators of exercise load. Seeing as how they are all supposedly measuring the same thing (at a high level) they should rise and fall in value together. It is possible to validate that these features have a linear relationship by calculating the correlation matrix for these columns.

In [ ]:
labels_train = train_set[["%HRmax", "%HRres", "Blood lactate", "RPE (CR10)"]].copy()
for index, row in labels_train.iterrows():
    if row["%HRmax"] == 0 or row["%HRres"] == 0 or row["Blood lactate"] == 0 or row["RPE (CR10)"] == 0:
      labels_train.drop(index, axis=0, inplace=True)

labels_correlation = labels_train.corr()
print(labels_correlation)

                 %HRmax    %HRres  Blood lactate  RPE (CR10)
%HRmax         1.000000  0.992338       0.600295    0.191919
%HRres         0.992338  1.000000       0.502976    0.279248
Blood lactate  0.600295  0.502976       1.000000   -0.399668
RPE (CR10)     0.191919  0.279248      -0.399668    1.000000


From this exercise, one can see that max HR and res HR have a nearly perfect linear relationship. Thus, we can find the line of best fit between these featuers and fill in missing data accordingly.

The relationship between %HRmax and blood lactate is less strictly linear, but this method should still serve well enough for now.

# Linear Regression for %HRmax, %HRres, and Blood lactate Categories
To fill in the missing data for the HRmax, HRres, and Blood lactate categories, I intend to calculate the best fit line between these variables. These linear models will enable me to fill in the missing data in a way that is more precise than simply using the median or mean values.

In [ ]:
lin_regression_max_dependent = stats.linregress(labels_train["%HRmax"], labels_train["%HRres"])
lin_regression_res_dependent = stats.linregress(labels_train["%HRres"], labels_train["%HRmax"])
lin_regression_max_dependent_blood_lactate = stats.linregress(labels_train["%HRmax"], labels_train["Blood lactate"])
lin_regression_blood_lactate_dependent_max = stats.linregress(labels_train["Blood lactate"], labels_train["%HRmax"])

for index, row in train_set.iterrows():
    if (row["%HRres"] == 0 and row["%HRmax"] != 0):
      train_set.at[index, "%HRres"] = lin_regression_max_dependent.intercept + lin_regression_max_dependent.slope * row["%HRmax"]
    if (row["%HRmax"] == 0 and row["%HRres"] != 0):
      train_set.at[index, "%HRmax"] = lin_regression_res_dependent.intercept + lin_regression_res_dependent.slope * row["%HRres"]
    if (row["Blood lactate"] == 0 and row["%HRmax"] != 0):
      train_set.at[index, "Blood lactate"] = lin_regression_max_dependent_blood_lactate.intercept + lin_regression_max_dependent_blood_lactate.slope * row["%HRmax"]
    if (row["Blood lactate"] != 0 and row["%HRmax"] == 0):
      train_set.at[index, "%HRmax"] = lin_regression_blood_lactate_dependent_max.intercept + lin_regression_blood_lactate_dependent_max.slope * row["Blood lactate"]
    #I threw the first if statement in again at the end to make up for the fact that new values may have been added to the HRmax feature
    if (row["%HRres"] == 0 and row["%HRmax"] != 0):
      train_set.at[index, "%HRres"] = lin_regression_max_dependent.intercept + lin_regression_max_dependent.slope * row["%HRmax"]


# Filling Data for RPE (CR10)
Because there is little to no linear relationship between RPE (CR10) and any of the other labels, I will use the mean value of this column for all missing values in the column.

In [ ]:
mean = train_set[train_set["RPE (CR10)"] != 0]["RPE (CR10)"].mean()
train_set["RPE (CR10)"].replace(0, mean, inplace = True)

Any record that is still missing data in the output feature columns are dropped entirely from the training set.

In [ ]:
for index, row in train_set.iterrows():
  if row["%HRmax"] == 0 or row["%HRres"] == 0 or row["Blood lactate"] == 0 or row["RPE (CR10)"] == 0:
    train_set.drop(index, axis = 0, inplace=True)

# Defining and Training the Model
In this step, I used the Tensorflow.Keras package to define a neural network that features two hidden layers and uses relu activation. I chose mean absolute percentage error for the loss function, as the labels are of varying scales. Using the absolute percent error should help to weight loss on each of the labels equally and ignore their differences in scale.

###Cross-Validation
To verify the functionality of the model without contaminating the test data that has been set aside, I used 4-fold cross-validation. This process allows me to tune the architecture and any hyperparameters at will without worrying about fitting to the test data.

In [ ]:
train_set.dropna(inplace=True)
model = Sequential()
model.add(Dense(500, input_dim = 11, activation="relu"))
model.add(Dense(250, activation="relu"))
model.add(Dense(4, activation="linear"))
model.compile(loss="mean_absolute_percentage_error", optimizer="adam")
model.save_weights("initial")

KFold_loss = []
kf = KFold(n_splits = 4)
kf.get_n_splits(train_set)
for train_index, test_index in kf.split(train_set):
  model.load_weights("initial")
  X_train, X_test = train_set.iloc[train_index, :11], train_set.iloc[test_index, :11]
  y_train, y_test = train_set.iloc[train_index, 11:], train_set.iloc[test_index, 11:]
  model.fit(X_train, y_train, epochs=400, batch_size=32)
  KFold_loss.append(model.evaluate(X_test, y_test))


Epoch 1/400
9/9 [==============================] - 1s 4ms/step - loss: 96.5921
Epoch 2/400
9/9 [==============================] - 0s 4ms/step - loss: 85.9878
Epoch 3/400
9/9 [==============================] - 0s 4ms/step - loss: 75.4215
Epoch 4/400
9/9 [==============================] - 0s 3ms/step - loss: 70.0458
Epoch 5/400
9/9 [==============================] - 0s 3ms/step - loss: 66.0452
Epoch 6/400
9/9 [==============================] - 0s 4ms/step - loss: 65.7370
Epoch 7/400
9/9 [==============================] - 0s 3ms/step - loss: 63.6827
Epoch 8/400
9/9 [==============================] - 0s 4ms/step - loss: 63.5616
Epoch 9/400
9/9 [==============================] - 0s 4ms/step - loss: 60.5284
Epoch 10/400
9/9 [==============================] - 0s 3ms/step - loss: 59.3066
Epoch 11/400
9/9 [==============================] - 0s 3ms/step - loss: 59.4276
Epoch 12/400
9/9 [==============================] - 0s 3ms/step - loss: 55.9699
Epoch 13/400
9/9 [==============================]

# Testing the Model
The final step of this process is to test the model on the training data that was set aside. The metric I used to quantify the model's performance on the test data is the mean absolute percent error. This is calculated by finding the absolute percent error for each label within each example, taking the average of these values, and then averaging these averages over the entire test data set.

$(\frac{1}{examples}\sum\limits_{j=1}^{j=examples}(\frac{1}{labels}\sum\limits_{n=1}^{n=labels}|\frac{Y_{predicted} - Y_{actual}}{Y_{actual}}|)) * 100$ = MAPE

This method allows me to ignore instances in the test set where label data is missing; I only calculate the absolute percent error for labels that have data present.

In [ ]:
test_set.dropna(inplace=True)
whole_set_mean_absolute_percent_error = 0

for index, row in test_set.iterrows():
  absolute_percent_error_single_example = 0
  test_example_np = row.iloc[:11].to_numpy().reshape(1,11)
  prediction = model.predict(test_example_np)
  num_non_zero_labels = 0
  for label in range(0,len(prediction[0])):
    if row.iloc[11+label] == 0:
      continue
    else:
      num_non_zero_labels += 1
      absolute_percent_error_single_example += abs((prediction[0][label] - row.iloc[11+label])/row.iloc[11+label] * 100)
  whole_set_mean_absolute_percent_error += absolute_percent_error_single_example/num_non_zero_labels

whole_set_mean_absolute_percent_error/=test_set.shape[0]
print("Test Set Mean Absolute Percent Error: " + str(whole_set_mean_absolute_percent_error) + "%")
     



Test Set Mean Absolute Percent Error: 12.770439719516208%
